In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

21/12/01 06:37:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
import os

def getFolderFilePaths(checkLocation, pathToLocation = ''):
    paths = list()
    checkPath = os.path.join(pathToLocation, checkLocation)
    if os.path.isfile(checkPath):
        if (checkLocation.find('.') == 0):
            return paths
        paths.append(checkPath)
        return paths
    else:
        for innerFile in os.listdir(checkPath):
            innerPaths = getFolderFilePaths(innerFile, checkPath)
            paths.extend(innerPaths)
    return paths
    

In [3]:
def getFilePathsFromFolders(folders):
    all_paths = []
    for folder in folders:
        folder_paths = getFolderFilePaths(folder)
        all_paths.extend(folder_paths)
    return all_paths

In [ ]:
from operator import add

def lower_clean_str(x):
    alpha='ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    lowercased_str = x.upper()
    for ch in x:
        if ch not in alpha:
            lowercased_str = lowercased_str.replace(ch, '')
    return lowercased_str
    

def createInvertedIndex():
    stop_list = {'they', 'she', 'he', 'it', 'the', 'as', 'is', 'and', ''}
    folders = ['shakespeare', 'Hugo', 'Tolstoy']
    paths = getFilePathsFromFolders(folders)
    wordCountTotal = None
    for file_path in paths:
        
        # read data from text file and split each line into words
        words = sc.textFile(file_path).flatMap(lambda line: line.split(" "))
        # count the occurrence of each word
        words = words.map(lower_clean_str)
        words = words.filter(lambda word: word not in stop_list)
        wordCounts = words.map(lambda word: (word, file_path))
        wordCounts = wordCounts.map(lambda wp: ((wp[0], wp[1]), 1)).reduceByKey(add)
        wordcount_paths = wordCounts.map(lambda w: (w[0][0], [(w[0][1], w[1])]))
        if wordCountTotal == None:
            wordCountTotal = wordcount_paths
        else:
            wordCountTotal = wordCountTotal.union(wordcount_paths).reduceByKey(lambda x,y: x+y)
    
    wordCountTotal.saveAsTextFile("/notebooks/14848_hw5_spark/results")

    
createInvertedIndex()